# **Tutorial 4** - Submitting Your Solution

You learned about AI agents in the previous tutorial and about the data in [tutorial 2](https://github.com/cg-gdsc/GDSC-7/blob/main/tutorials/Tutorial_2_Data_Understanding.ipynb). Now, it's time to combine this knowledge into one working solution! 
Unlike in previous GDSCs where you were just asked to hand in the results of your model, this year we ask you to submit running `source code`.
I.e. you will create a full blown chatbot API. We will then run your code (What could possibly go wrong?), and ask it to answer a set of questions.
The answers to these questions will be stored in a database and used in the [Chatbot Arena](https://gdsc.ce.capgemini.com/app/arena/) where you (and everybody else) will vote for the best solution!

In this tutorial, we will work with the `BasicPIRLSCrew` from [tutorial 3](https://github.com/cg-gdsc/GDSC-7/blob/main/tutorials/Tutorial_3_Introduction_to_AI_Agents.ipynb). You will learn how to 
- turn our Juypter Notebook solution into a real chatbot,
- test your code both locally and remotely using AWS, and
- submit your solution to compete with other teams.

But this is not all! As part of this year's GDSC, you are also required to evaluate the submissions of other teams. At the end of this tutorial, we will explain why your role as evaluators is essential and how you can participate in the [Chatbot Arena](https://gdsc.ce.capgemini.com/app/arena/)!

# Agenda
1. [Understanding the project structure](#understanding-the-project-structure) - explains the reasoning behid the code structure and which files are crutial for your submission.
2. [Create your first application](#create-your-first-application) - covers the steps to run your first app.
3. [How to test your code](#how-to-test-your-code) - shows you how to run tests both locally and remotely.
4. [How to submit your code](#how-to-submit-your-code) - shows you how to finally submit your solution to the competition.
5. [How does the evaluation work](#how-does-the-evaluation-work) - explains the logic of automatic evaluation.
6. [How to check the status of your application](#how-to-check-the-status-of-your-application) - how to access the logs.
7. [What to do if the automatic evaluation fails](#what-to-do-if-the-automatic-evaluation-fails)
8. [Chatbot Arena](#chatbot-arena) - shows you how to use the Arena and explains why it is important to rate battles.
9. [Human evaluation questions](#human-evaluation-questions) - shows you how to add new questions to the competition and explains the benefits of it.

# Understanding the project structure

Before we get to the code, let's take a step back and review the code structure:

## [Code Commit](https://aws.amazon.com/de/codecommit/)
When you created or joined a team, you gained access to an AWS account. From the AWS Management Console, you can navigate to AWS CodeCommit to view your team's repository.

1. [<img src="../images/t4_code_commit_1.png" width=800/>](../images/t4_code_commit_1.png) $\space\space\space\space$ 2. [<img src="../images/t4_code_commit_2.png" width=800/>](../images/t4_code_commit_2.png)

In CodeCommit, you can see your team's repository with code that has already been prepared for you. We will explain this code in more detail throughout this and the next tutorial.

## Code structure

In your [CodeCommit repository](https://aws.amazon.com/de/codecommit/), that you should also see on the left hand side of this notebook, you will find three folders:

- `images` - Containing all the images used in the tutorials
- `src` - Containing the source code you will actually submit
- `tutorials` - The trainings that you are currently going through

In the `src` folder, there are two main directories: `submission` and `static`.

- `submission` - **This is the directory where all of your code will be placed**. Here, you can modify and create new crews, tools, agents, and do whatever your heart desires.
- `static` - This directory contains code that cannot be modified. The static directory is replaced by the GDSC team with each submission, so no changes made here will be reflected in your final submission.

Despite the considerable freedom given to participants in the submission directory, there is one particularly important file to which you must pay extra attention:  [src/submission/create_submission.py](../src/submission/create_submission.py). Let's take a look inside!

```python
from src.static.ChatBedrockWrapper import ChatBedrockWrapper
from src.static.submission import Submission

def create_submission(call_id: str) -> Submission:
    ...
```

As you can see, there's a single function defined. This function will be **the entry point** for your submission. It is used to instantiate your submission, and the signature of this function cannot be modified. It must take a string named `call_id` and return an object of type `Submission`. The body of the function and necessary imports are up to your implementation.

### What is `Submission`
`Submission` is an abstract class that enforces the implementation of a run method for your solution. This is important because we expect your submission to have a method with that name, which accepts a `str` and returns a `str`. This method is how your crew will receive the question (prompt) and how we (the GDSC team) expect to get the answer.

Implementation of this class can be found in the [src/static/submission.py](src/static/submission.py)

```python
from abc import ABC, abstractmethod

class Submission(ABC):
    @abstractmethod
    def run(self, prompt: str) -> str:
        ...
```

As you can see, this file contains nothing more than the abstract class Submission. It serves as a useful interface that enforces your submissions to implement the run method.

It's worth mentioning that due to Python's dynamic typing, any object with a method named `run` that satisfies the signature will work. However, this abstract class is a good programming practice because it clearly defines the expected interface for your submissions.

## Create your first submission
Let's go back to the `BasicPIRLSCrew` defined in the previous tutorial. We already extracted the code into a Python script. The code can be found in the [src/submission/crews/basic_PIRLS_crew.py](../src/submission/crews/basic_PIRLS_crew.py). 
Note how the `BasicPIRLSCrew` inherits from the `Submission` class and implements the `run` function.

```python
# ... code cut out

class BasicPIRLSCrew(Submission):  # BasicPIRLSCrew inherits from Submission class

    def __init__(self, llm: ChatBedrockWrapper):
        self.llm = llm

    def run(self, prompt: str) -> str:
        return self.crew().kickoff(inputs={"prompt": prompt}).raw

# ... code cut out
```


Additionally, the `query_database` tool is now in [src/submission/tools/database.py](src/submission/tools/database.py). Slightly changed to not return output that is too long. 
(Having the output of a `SELECT * FROM STUDENTS` as input to your LLM is a surefire way to spend all your money and crashing the system)

Now that our `BasicPIRLSCrew` class is ready, we can modify the `create_submission` function so that it returned our new class.

```python
from src.submission.crews.basic_PIRLS_crew import BasicPIRLSCrew
from src.static.ChatBedrockWrapper import ChatBedrockWrapper
from src.static.submission import Submission

def create_submission(call_id: str) -> Submission:
    llm = ChatBedrockWrapper(
        model_id='anthropic.claude-3-haiku-20240307-v1:0',
        model_kwargs={'temperature': 0},
        call_id=call_id
    )
    crew = BasicPIRLSCrew(llm=llm)  # instantiate the new class
    return crew
```


One important point to note is the `ChatBedrockWrapper` class, which is used as the llm argument for your crew. This class is an extension of the [`ChatBedrock`](https://python.langchain.com/v0.2/docs/integrations/chat/bedrock/) class you used in the previous tutorial, and it handles communication with [AWS Bedrock](https://aws.amazon.com/bedrock/). The main difference is that it requires an additional argument, `call_id`, due to a technical requirement in this year's GDSC. The key point is that you **must** use this class instead of the standard ChatBedrock. This wrapper provides direct access to the number of tokens used by your submission. (See [src/static/ChatBedrockWrapper.py](../src/static/ChatBedrockWrapper.py))

While you're not required to use crewAI, you **must** use this wrapper for all your llm interactions, as it includes the implementation for token counting and cost tracking.

# How to test your code?
There are two ways to test your code:
- In AWS by pushing it to the test branch
- On your local machine

We go over the AWS method now. The local version is explained in the appendix.

# How to push to the submission branch?

First, we need to push our changes to the `test_submission` branch. To do that open the terminal in your JupyterLab (go to SageMaker and click on "Notebooks" in the menu on the left to see your team's notebooks).

[<img src="../images/t4_console_1.png"  width=1200/>](../images/t4_console_1.png)

Now that you have access to the terminall we can focus on a few basic git actions:
1. Check if our code is up to date by using `git pull`.
2. See what branches do we have currently. Use `git branch`.
3. In order to test our code we need to push changes to the **test_submission** branch. Create it by using `git checkout -b test_submission`. Note that the name of the branch is important and it has to be called **test_submission**. The same way you can create your **submission** branch.
4. After you make any changes you can see the status of your current branch by using `git status`. If you want to add a specific file to the commit you can use `git add name_of_the_file` and if you want to add all files at once simply use `git add .`.
5. Create a commit with a descriptive message using `git commit -m "descriptive message of all changes"`.
6. Push your changes using `git push`. (The first time on the new branch you may need to use `git push --set-upstream origin test_submission`)

If you haven't work with *git* before it's worth reading up on it. [Here](https://www.w3schools.com/git/git_intro.asp?remote=github) is a good place to get started.

You can also push your changes using CodeCommit service directly. Firt, be sure you are on the proper branch and then introduce the changes. When ready fill the *Author name* and *Email address* (Commit message is optional) and pres the *Commit changes* button.

[<img src="../images/t4_code_commit_push_1.png"  width=1200/>](../images/t4_code_commit_push_1.png)


Pushing changes to the **test_submission** branch will automatically start a pipeline that will create a docker image containing the code that you just pushed, and start the app using the `app.py` script. 
You should soon (< 2 min) see the results on the [Elastic Container Service (ECS)](https://us-east-1.console.aws.amazon.com/ecs/v2/clusters?region=us-east-1): go to the clusters and select the gdsc cluster, select the test service, and go to the taks to see its public IP address. 

[<img src="../images/t4_ecs_1.png"  width=1200/>](../images/t4_ecs_1.png)

[<img src="../images/t4_ecs_2.png" width=1200/>](../images/t4_ecs_2.png)

Now that we have the IP of our new submission we can send requests to this endpoint. Let's test it!

In [ ]:
import requests

# Some wrapper code to send requests to our crew
def ask_question(question: str, url: str):
    data = {'prompt': question}
    headers = {'Content-Type': 'application/json'}
    response = requests.post(url, json=data, headers=headers)

    return response.json()

In [ ]:
REMOTE_HOST = 'http://ADD_YOUR_PUBLIC_IP_HERE/run'

In [ ]:
res = ask_question("How many students participated in the study?", REMOTE_HOST)
print(res['result'])

If things go well you will see an answer after a few seconds. If it doesn't work, keep on reading. We'll cover how to debug errors next.

Here is a list of values returned by your submission:
- result - it is the answer retuend by your crew as a result of calling the `run` method.
- time - time it took for your crew to answer the question in seconds.
- timed_out - information about whether your submission timed out or not.
- tokens - overall number of tokens used in all agents' converstions to generate the final answer.
- cost - what was the cost for your submission for getting the answer.
- token_details - this is a dictionary that holds more detailed data about token usage. Here you can see how many prompt and completion tokens used each of the models you selected.

# How to submit your code?
Submitting your code is similar to testing it using AWS. This time, however, you will be pushing your changes to the `submission` branch. Like the `test_submission` process, this initiates the `app.py` script in the Docker instance, but it doesn't stop there. An automatic evaluation is run to test your submission, and you can monitor the evaluation status on your team's page. It's basically a test that ensures that your code works and can answer basic questions. After successfull evaluation your submission will be eligible for to the competition and compete in the chatbot arena!

# How does the evaluation work?
After submitting your code it is tested using a few automatic evaluation questions. Your submission has to yield correct answers, and the response time should be shorter than the specified timeout. If either of these conditions is not fullfilled for any of the automatic evaluation questions, your submission is not allowed to participate any further. Nonetheless, it still affects the total number of submissions your team has made.

We all know how unstable LLMs tend to be. This is why each automatic evaluation question is asked 3 times, and in the worst case scenario, the total evaluation can take up to 1 hour! Be patient and you'll see your results in your [team's page](https://gdsc.ce.capgemini.com//app/portal/).

During the evaluation your submission will look like this:

[<img src="../images/t4_submission_status_1.png" width=1200/>](../images/t4_submission_status_1.png)

After a while the status should be updated. In this case we see that automatic evaluation did not pass the reason being that our crew did not answer the questions correctly. :( 

[<img src="../images/t4_submission_status_2.png" width=1200/>](../images/t4_submission_status_2.png)

But actually, that is something that we could have expected if you tested the other questions in tutorial 3. Our current solution does not have the full overview of the database and hence is unable to answer at to all of the automatic questions correctly. The possible evaluation fail reasons are explained in the section [What to do if the automatic evaluation fails](#what-to-do-if-the-automatic-evaluation-fails). We'll build a better solution in the next tutorial.

Note that even if the submission didn't pass it is still included in the total number of submissions your team has done.

# How to check the status of your application?
Understanding what's going on during the evaluation is essential for debugging. Here's how:
- You can access the logs directly on ECS.
- Go to the test task and enter the currently running application.
- Click on the 'Logs' located on the top bar.
- If you want to access the logs in real-time, click on the button to CloudWatch.

The images illustrate the key steps.
The first step is the same as [here](#testing-using-aws)

[<img src="../images/t4_ecs_1.png" width=1200/>](../images/t4_ecs_1.png)

[<img src="../images/t4_log_1.png" width=1200/>](../images/t4_log_1.png)

[<img src="../images/t4_log_2.png" width=1200/>](../images/t4_log_2.png)

Logs are only enabled for the testing tasks and the ECS task will automatically **shut down after 30 minutes**. This should be enough time to run your tests but if you need more time, you would have to make a dummy change to the code and push your changes to the **test_submission** branch again.

# What to do if the automatic evaluation fails?
If the automatic evaluation fails that means your new submission did not answer correctly, timed out or another unexpected error occured. The reason is displayed on the the team's website:
- "Unexpected error" - this status means there were issues with starting the `app.py` script. Chceck your submission implementation and `create_submission` function implementation.
- "Timeout" - this status means that your submission failed to answer to at least one of automatic evaluation questions in time.
- "Incorrect answers" - this status means that at least one of the answers was not correct. 

If the displayed status is "In progress" for a suspiciously long period of time, that being a few hours, please contact GDSC organizers.

### How to react?
- If the status is an "Unexpected error" that means there is something wrong with your implementation. Run it locally to see if there are any issues with your code.
- In case of "Timeout" we do not know if the answers were correct. That only shows that your submission was taking a bit too long. Try improving your prompts. Maybe include better information about the database, add more tools or remove the unused ones or change the agents themselves?
- In case of "Incorrect answers" see if your crew has access to the PIRLS database. You could also check if the database information provided to the agents is sufficient enough.

In general reviewing the logs will help you understand what exactly your agents were doing and what it is that they're struggling with. Remember to first make sure your code runs in Sagemaker and then in the **test_submission** branch before submitting a new version.

We will show you how to improve your crew in the [tutorial 5](https://github.com/cg-gdsc/GDSC-7/blob/main/tutorials/Tutorial_5_Advanced_AI_Agents.ipynb).

# Chatbot Arena
Before we move on to building better agents, let understand how your solutions are evaluated. 
Evaluating LLM-based solutions is not a trivial task. There are no straightforward metrics, such as accuracy or F1 score, to easily compare different models. This is why most of the time, when dealing with LLM text output, human or semi-automated evaluation is used. This idea of human-based evaluation underlies the concept of the Chatbot Arena.

As a **human evaluator**, you can go to the [arena website](https://gdsc.ce.capgemini.com/app/arena/), 

[<img src="../images/t4_arena_1.png" width=800/>](../images/t4_arena_1.png)

choose a question from a list, and see how 2 random submissions responded to this question. 

[<img src="../images/t4_arena_2.png" width=800/>](../images/t4_arena_2.png)

Compare the results and decide which chatbot returned a better answer. Perhaps both are acceptable, or maybe both are complete nonsense? Below the text areas, select the appropriate verdict.

[<img src="../images/t4_arena_3.png" width=800/>](../images/t4_arena_3.png)

It's worth mentioning that everyone can evaluate questions, not just the people taking part in the GDSC.
It is also highly unlikely that you will evaluate the exact same battle twice.

**Your vote determines the rating of the submissions.** We use the [Glicko 2](https://en.wikipedia.org/wiki/Glicko_rating_system) rating system to determine how good the models are based on your votes!

#### Why should I rank battles?
We need human evaluators like you to assess the quality of the returned answers. Your decision is counted as a win or loss for the competing submissions, and their ranking is updated based on your opinion. Because this is such an important step, we (the GDSC Team) have introduced some constraints on how many submissions your team can make. The first two submissions require no additional effort. However, if your team wants to add more submissions, it is required to rank a specific number of battles. This number starts low for the initial submissions and increases over time to a steady value of 50 ranked battles per submission.

This number is a total for your team and not a requirement for a single team member.

This system had to be implemented because having a large number of submissions requires a lot of battles for adequate ranking. Because there is almost no automatic evaluation in this year's GDSC edition, it's best to submit solutions that are robust and have a real chance of competing with others, rather than focusing on minor improvements.

# Human Evaluation Questions
Right now, on the arena website, you can see a bunch of predefined questions that can be asked to assess the quality of submitted crews. Every submission that passes the automatic evaluation is asked this set of questions, and the responses are stored in the database. You can, however, submit your own questions - specifically tough ones that your crew implementation excels at! This could help your submission win more battles and climb the rankings, as long as it can also handle the existing questions. We're seeking a general solution, not a highly specialized one.

Before adding a new question, be sure there is no similar question already in the list. This list will change over time as new, interesting questions pop up either from the GDSC Team's end or from you and other participants.

**All questions added by participants will be verified before being added to the list.**


# Conclusions
In this tutorial, we walked you through the repository structure, highlighting the most critical parts of the code. Since the crew operates directly from your implementation, it's absolutely essential to stick to the required interfaces - like the `create_submission` function - so your crew doesn't go rogue and start causing mayhem.

We also showed you how to commit changes using both JupyterLab's console and CodeCommit service, and emphasized the significance of the two branches: test_submission and submission. Checking your submission status on the GDSC page and monitoring logs via ECS were also covered to help you stay on top of things.

Finally, we explained the role of the arena, how to participate in battles, and how the rating process works. You also learned about the evaluation questions, and that you can submit your own, especially if they highlight the strengths where your solution really shines.

Next up is our [last tutorial](https://github.com/cg-gdsc/GDSC-7/blob/main/tutorials/Tutorial_5_Advanced_AI_Agents.ipynb), where we show you how to improve your solution! Let's go!

# Appendix

## Working on a locally

If you are not working in AWS Sagemaker, there a few things to consider. Let's go over them.

### Checking out the code commit repo
If you want to use this code in your local IDE, you'll need to install git-remote-codecommit, set up credentials for your AWS account locally, and then use the HTTPS (GRC) clone URL to download the repo. A tutorial for that can be found [here](https://docs.aws.amazon.com/codecommit/latest/userguide/setting-up-git-remote-codecommit.html).


### Testing locally
Run the application locally and send a bunch of test requests to localhost url. To do that run this command in the terminal

#### Linux
```bash
# activate python venv if needed (assuming venv is the virtual enviroment direcotry)
source venv/bin/activate

# add the current directory to the python path
export PYTHONPATH="$PYTHONPATH:$(pwd)"

# run the application
python src/static/app.py
```

#### Windows
```bat
rem activate python venv if needed (assuming venv is the virtual enviroment direcotry)
venv\Scripts\activate

rem add the current direcotry to the python path
set PYTHONPATH=%PYTHONPATH%;%cd%

rem run the application
python src\static\app.py
```

#### Expected console output:
```
INFO:     Started server process [7956]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
```

Now we can see that our application runs on the localhost. To test it out we need to send a POST request to the localhost/run endpoint with a payload that contains the prompt. Here is an exaple code how to do that

In [ ]:
LOCAL_HOST = 'http://127.0.0.1:8000/run'
res = ask_question("How many students participated in the study?", LOCAL_HOST)
print(res['result'])